## Lab 1

### Part 2: Neural Machine Translation in the wild
In the third homework you are supposed to get the best translation you can for the **EN-RU** translation task.

Basic approach using RNNs as encoder and decoder is implemented for you. 

Your ultimate task is to use the techniques we've covered, e.g.

* Optimization enhancements (e.g. learning rate decay)

* CNN encoder (with or without positional encoding)

* attention/self-attention mechanism

* pretraining the language model

* [Byte Pair Encoding](https://github.com/rsennrich/subword-nmt)

* or just fine-tunning BERT ;)

to improve the translation quality. 

__Please use at least three different approaches/models and compare them (translation quality/complexity/training and evaluation time).__

Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

In [1]:
!pip install sacremoses
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 4.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=37bd1d32e7a8652ccc9567e390ab038b30cb3772230ade45c3f15ffd18a175f0
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 3.7 MB/s 
     |████████████████████████████████| 182 kB 1.7 MB/s 
     |████████████████████████████████| 7.6 MB 21.4 MB/s 


In [2]:
import os
path_do_data = '../../datasets/Machine_translation_EN_RU/data.txt'
if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github. Loading special files as well")
    !wget https://raw.githubusercontent.com/girafe-ai/ml-course/master/datasets/Machine_translation_EN_RU/data.txt -nc
    path_do_data = './data.txt'

Dataset not found locally. Downloading from github. Loading special files as well
--2022-12-13 11:02:22--  https://raw.githubusercontent.com/girafe-ai/ml-course/master/datasets/Machine_translation_EN_RU/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12905334 (12M) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  12.31M  --.-KB/s    in 0.1s    

2022-12-13 11:02:22 (123 MB/s) - ‘data.txt’ saved [12905334/12905334]



In [3]:
import random
import math
import time

import tqdm
import pandas as pd

In [4]:
data = pd.read_csv('data.txt', sep='\t', names=['ENG','RUS'])

In [5]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

In [6]:
tokenizer = FSMTTokenizer.from_pretrained("facebook/wmt19-ru-en")
model = FSMTForConditionalGeneration.from_pretrained("facebook/wmt19-ru-en")

Downloading:   0%|          | 0.00/776k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/639k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

In [8]:
source_txt = data['RUS'][:50]
tok_encode = [tokenizer.encode(sent, return_tensors="pt") for sent in source_txt]
outputs = []
for i in tqdm.tqdm(range(50)): # range(len(tok_encode))    
    outputs.append(model.generate(tok_encode[i]))
result_txt = [tokenizer.decode(output[0], skip_special_tokens=True) for output in outputs]
print(result_txt)

100%|██████████| 50/50 [02:12<00:00,  2.65s/it]

['Hotel Cordelia is located in Tbilisi, a 3-minute walk from Holy Trinity Cathedral.', 'Facilities at the Tupirmarka Lodge include a 24-hour front desk and a snack bar.', 'Located in Shanghai, Apartments Naigao Xiaowo offers free Wi-Fi in all areas.', 'There is also a TV and a private bathroom with shower.', 'This air-conditioned room comes with satellite TV.', 'They are decorated in cream tones, some with private balconies with a living area.', 'Staff at reception are happy to help guests plan excursions.', 'Some rooms have a seating area.', 'Niumarkt Square, the Kalverstraat shopping street, the Hermitage Museum on Amstel and Rembrandt Square are all less than a 15-minute walk away.', 'The Danube Delta is a nature reserve where you can go fishing, watch birds, take a boat ride, or simply relax in nature.', 'Rooms are equipped with air conditioning, a flat-screen TV with cable channels, a work desk, a fan, and a seating area.', 'Each apartment comes with a balcony, a safe and a flat-s

In [9]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction

In [11]:
source = []
for i in data['ENG'][:50]:
    source.append(i.split())

result = []
for j in result_txt:
    result.append(j.split())

corpus_bleu([[text] for text in source], result) * 100

31.16455577990067

# Corpus BLEU for pretrained model = 31.16
(facebook/wmt19-en-ru from https://huggingface.co/facebook/wmt19-en-ru )

Baseline solution BLEU score is quite low. Try to achieve at least __24__ BLEU on the test set. 
The checkpoints are:

* __22__ - minimal score to submit the homework, 30% of points

* __27__ - good score, 70% of points

* __29__ - excellent score, 100% of points